In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from delta.tables import *

In [0]:
df_silver = spark.read.format("delta").option('header','true').option('path','abfss://silver@azadlsgen7428.dfs.core.windows.net/car_sales').load()

In [0]:
display(df_silver)

In [0]:
df_dealer = spark.sql('select * from cars_catalog.gold.dim_dealer')
df_branch = spark.sql('select * from cars_catalog.gold.dim_branch')
df_model = spark.sql('select * from cars_catalog.gold.dim_model')
df_date = spark.sql('select * from cars_catalog.gold.dim_date')

In [0]:
df_fact = df_silver.join(df_dealer, df_silver.Dealer_ID == df_dealer.Dealer_ID, 'left').join(df_branch, df_silver.Branch_ID == df_branch.Branch_ID, 'left').join(df_model, df_silver.Model_ID == df_model.Model_ID, 'left').join(df_date, df_silver.Date_ID == df_date.Date_ID, 'left').select(df_silver.Revenue,df_silver.Units_Sold,df_silver.Revenue_Per_Unit,df_dealer.dim_dealer_key,df_branch.dim_branch_key,df_model.dim_model_key,df_date.dim_date_key)

In [0]:
display(df_fact)

In [0]:
if spark.catalog.tableExists("cars_catalog.gold.fact_sales"):
    delta_table = DeltaTable.forName(spark,'cars_catalog.gold.fact_sales')
    delta_table.alias('t').merge(df_fact.alias('s'),'t.Date_ID = s.Date_ID,t.dim_dealer_key = s.dim_dealer_key,t.dim_branch_key = s.dim_branch_key,t.dim_model_key = s.dim_model_key').whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
else:
    df_fact.write.format("delta").mode("append").option('path','abfss://gold@azadlsgen7428.dfs.core.windows.net/fact_sales').saveAsTable("cars_catalog.gold.fact_sales")


In [0]:
%sql

select * from cars_catalog.gold.fact_sales